## Código utilizado para análise de dados da reportagem: [Eleições 2024: PT e PL estão no mesmo palanque de candidatos a prefeito em 85 cidades no país](https://g1.globo.com/politica/eleicoes/2024/eleicao-em-numeros/noticia/2024/08/16/eleicoes-2024-pt-e-pl-pelo-pais.ghtml)

In [1]:
# Importa as bibliotecas necessárias para a análise
import pandas as pd

In [2]:
# Me mostre tudo
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
# Lê os dados do arquivo CSV contendo informações eleitorais
df = pd.read_csv("data/consulta_cand_2024_BRASIL.csv", delimiter =';', encoding='latin-1')

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\judite\AppData\Local\Temp\ipykernel_1284\2737666020.py:2: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv("data\consulta_cand_2024_BRASIL.csv", delimiter =';', encoding='latin-1')
C:\Users\judite\AppData\Local\Temp\ipykernel_1284\2737666020.py:2: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv("data\consulta_cand_2024_BRASIL.csv", delimiter =';', encoding='latin-1')


FileNotFoundError: [Errno 2] No such file or directory: 'data\\consulta_cand_2024_BRASIL.csv'

In [ ]:
df.head()

### 🧹 Limpando duplicados

In [ ]:
# Checando os duplicados segundo o número de identificação do título de eleitor
duplicados = df[df.duplicated(subset='NR_TITULO_ELEITORAL_CANDIDATO', keep=False)]

In [ ]:
# Apagando duplicados
df = df.drop_duplicates(subset='NR_TITULO_ELEITORAL_CANDIDATO')

### 🗺️ Add código IBGE

In [ ]:
# União com o df de códigos do IBGE
codigo_ibge = pd.read_csv("data/municipios_brasileiros_tse.csv", delimiter =',', encoding='utf-8')

In [ ]:
codigo_ibge.head()

In [ ]:
df = pd.merge(codigo_ibge, df, left_on='codigo_tse', right_on='SG_UE', how='inner')

In [ ]:
df.head()

In [ ]:
# Selecionando apenas prefeitos
prefeito = df[df["CD_CARGO"] == 11]

In [ ]:
prefeito.head()

In [ ]:
prefeito["SG_PARTIDO"].unique()

## 🟥 Análise do Partido dos Trabalhadores (PT)

In [ ]:
# Filtra o DataFrame para coligações que contenham o partido específico
df_pt = prefeito[
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPT\b', na=False)
]

In [ ]:
# Conta a frequência dos valores na coluna especificada
frequencias = df_pt["TP_AGREMIACAO"].value_counts()
print(frequencias)

### ➡️ Coligações com o PT sem o PL

In [ ]:
# Situações em que aparecem o PT sem o PL:
df_PT_SEM_PL = prefeito[
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPT\b', na=False) &
    ~prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPL\b', na=False)
]

In [ ]:
frequencias = df_PT_SEM_PL["TP_AGREMIACAO"].value_counts()
print(frequencias)

## 🟩 Análise do Partido Liberal (PL)

In [ ]:
# Filtra o DataFrame para coligações que contenham o partido específico
df_pl = prefeito[
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPL\b', na=False)
]

In [ ]:
# Conta a frequência dos valores na coluna especificada
frequencias = df_pl["TP_AGREMIACAO"].value_counts()
print(frequencias)

### ➡️ Coligações com o PL sem o PT

In [ ]:
# Situações em que aparecem o PL sem o PT:
df_PL_SEM_PT = prefeito[
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPL\b', na=False) & 
    ~prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPT\b', na=False)
]

In [ ]:
frequencias = df_PL_SEM_PT["TP_AGREMIACAO"].value_counts()
print(frequencias)

## 🟩🟥 Coligações com o PT e o PL

In [ ]:
df_pt_pl = prefeito[
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPT\b', na=False) & 
    prefeito['DS_COMPOSICAO_COLIGACAO'].str.contains(r'\bPL\b', na=False)
]

In [ ]:
frequencias = df_pt_pl["TP_AGREMIACAO"].value_counts()
print(frequencias)

In [ ]:
df_pt_pl.head()

In [ ]:
coligações_ufs = df_pt_pl.groupby(["SG_UF"]).count() #sum function

In [ ]:
coligações_ufs

### ➡️ Dentre as coligações em que ambos estão, quem é a cabeça da chapa?

In [ ]:
# Conta a frequência dos valores na coluna especificada
df_pt_pl["SG_PARTIDO"].value_counts()

In [ ]:
# Salva o df
df_pt_pl.to_csv('data/coligacoes_pt_pl.csv', decimal=";", index=False, encoding='latin-1')